In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error

# Preparation for statistics modelling

In [2]:
pu_2019 = pd.read_csv('../data/curated/pu_2019.csv')
do_2019 = pd.read_csv('../data/curated/pu_2020.csv')
pu_2020 = pd.read_csv('../data/curated/pu_2020.csv')
do_2020 = pd.read_csv('../data/curated/do_2020.csv')

# EWR Airport has NaN population, we will fill 0 \
# because airport should have very few population
all_data = [pu_2019, do_2019, pu_2020, do_2020]
for data in all_data:
    data.fillna(0, inplace = True)

In [3]:
def print_model_performance(true, pred, model):
    """
    print the summary, anova table and rmse of a given linear model
    """
    print(model.summary())
    print("=" * 78)
    anova_table = sm.stats.anova_lm(model, robust = "hc2")
    print("ANOVA TABLE")
    print(anova_table)
    print("=" * 78)
    print('RMSE = {:<.4}'.format(np.sqrt(mean_squared_error(true, pred))))
    
    
def stepwise_selection(data, response, all_covariates, __covariates=None, __min_aic=np.inf, counter=1):
    """
    print the model attributes with lowest aic
    """
    print(f"Stage {counter}")
    before_aic = __min_aic
    min_aic = __min_aic
    if __min_aic == np.inf:
        __covariates = all_covariates
    # Remove one covariate and check the aic 
    for var in __covariates:
        new_covariates = [x for x in __covariates if x != var]
        formula = response + '~' + '+'.join(new_covariates)
        curr_model = ols(formula = formula, data = data).fit()
        print(f"{new_covariates}: {curr_model.aic}")
        if curr_model.aic < min_aic:
            min_aic = curr_model.aic
            best_model = curr_model
    # Add one covariate and check the aic
    for var in all_covariates:
        if var in __covariates:
            continue
        new_covariates = list(__covariates) + [var, ]
        formula = response + '~' + '+'.join(new_covariates)
        curr_model = ols(formula = formula, data = data).fit()
        print(f"{new_covariates}: {curr_model.aic}")
        if curr_model.aic < min_aic:
            min_aic = curr_model.aic
            best_model = curr_model
    if before_aic == min_aic:
        print(f"Best of Final: {list(__covariates)} with aic = {before_aic}\n")
    else:
        __covariates = best_model.params.index[1:]
        print(f"Best of Stage {counter}: {list(__covariates)} with aic = {best_model.aic}")
        return stepwise_selection(data, response, all_covariates, __covariates, min_aic, counter=counter+1)

# Modelling

## How does weather impact the number of trips?

In [4]:
stepwise_selection(pu_2019, 
                   'trip_count_in_month', 
                   ['feelslike', 'precip', 'windspeed', 'visibility', 'snow', 'snowdepth']
                   )
model1 = ols(formula = "trip_count_in_month ~ precip + windspeed + visibility + snow + snowdepth",
             data = pu_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model1.predict(pu_2020)
print_model_performance(pu_2020['trip_count_in_month'], trip_count_pred, model1)

Stage 1
['precip', 'windspeed', 'visibility', 'snow', 'snowdepth']: 81435.02916548605
['feelslike', 'windspeed', 'visibility', 'snow', 'snowdepth']: 81892.35310940816
['feelslike', 'precip', 'visibility', 'snow', 'snowdepth']: 84690.11198932484
['feelslike', 'precip', 'windspeed', 'snow', 'snowdepth']: 83413.67931524539
['feelslike', 'precip', 'windspeed', 'visibility', 'snowdepth']: 86702.8199638448
['feelslike', 'precip', 'windspeed', 'visibility', 'snow']: 87489.39197811416
Best of Stage 1: ['precip', 'windspeed', 'visibility', 'snow', 'snowdepth'] with aic = 81435.02916548605
Stage 2
['windspeed', 'visibility', 'snow', 'snowdepth']: 81912.58461788866
['precip', 'visibility', 'snow', 'snowdepth']: 88304.25917572824
['precip', 'windspeed', 'snow', 'snowdepth']: 83413.38663183774
['precip', 'windspeed', 'visibility', 'snowdepth']: 86937.71699249254
['precip', 'windspeed', 'visibility', 'snow']: 87511.92094083219
['precip', 'windspeed', 'visibility', 'snow', 'snowdepth', 'feelslike']: 

## How do populations and property prices impact the pick-up number in a zone?

In [5]:
stepwise_selection(pu_2019, 
                   'trip_count_total', 
                   ['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 
                    'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']
                   )

model2 = ols(formula = "trip_count_total ~ Price_per_square_feet \
                        + Population_By_LocationID + Density_per_hectare \
                        + ln_Price_per_square_feet + ln_Population_By_LocationID",
             data = pu_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model2.predict(pu_2020)
print_model_performance(pu_2020['trip_count_total'], trip_count_pred, model2)

Stage 1
['Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 90313.80632220773
['Price_per_square_feet', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 90403.5455995837
['Price_per_square_feet', 'Population_By_LocationID', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 90579.29825345267
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 90234.52737446017
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Density_per_hectare']: 90233.25976031511
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID']: 90233.07929795384
Best of Stage 1: ['Price_per_square_feet', 'Population_By_LocationID

## How do populations and property prices impact the drop-off number in a zone?

In [6]:
stepwise_selection(do_2019, 
                   'trip_count_total', 
                   ['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 
                    'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']
                   )

model3 = ols(formula = "trip_count_total ~ Population_By_LocationID + Density_per_hectare",
             data = do_2019
            ).fit(cov_type = 'HC2')

trip_count_pred = model3.predict(do_2020)
print_model_performance(do_2020['trip_count_total'], trip_count_pred, model3)

Stage 1
['Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 13103.167609860033
['Price_per_square_feet', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 13128.468292195046
['Price_per_square_feet', 'Population_By_LocationID', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 13154.127982278043
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Population_By_LocationID', 'ln_Density_per_hectare']: 13101.853478709972
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Density_per_hectare']: 13102.335683218664
['Price_per_square_feet', 'Population_By_LocationID', 'Density_per_hectare', 'ln_Price_per_square_feet', 'ln_Population_By_LocationID']: 13101.76226525108
Best of Stage 1: ['Price_per_square_feet', 'Population_By_Loca